# Ensemble learning
Notebook to explore improvement in performance of building custom-made ensemble learners. The workflow is the following:
1. Build-up of the individual learners and performance evaluation
- LighGBM
- XGBoost
- Random forest
- (Maybe) lasso regression
In this part, we will also include the resampling of data performing upsampling + downsampling

2. Study on how to ensemble them together for performance optimization
- Hard voting (including predictive threshold performance optimization for all of them)
- Soft voting (with a posterior predictive threshold optimization)
- Stacking



In [141]:
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBClassifier, plot_importance
from tqdm import tqdm
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
import lightgbm as lgbm

from auxFuns.EDA import *
from auxFuns.modelling import *


In [118]:
import auxFuns.modelling
importlib.reload(auxFuns.modelling)

<module 'auxFuns.modelling' from 'c:\\Users\\angel\\Documents\\VSCode\\rsv_modelling_transfer_learning\\auxFuns\\modelling.py'>

In [157]:
raw_datasets_path = os.getcwd() + '/datasets/raw'
processed_datasets_path = os.getcwd() + '/datasets/processed'

rsv_predictors_df_v2 = pd.read_csv(processed_datasets_path + '/rsv_predictors_phase1_daysDedup_seasons_prevTest.csv',low_memory=False)
rsv_predictors_df_v2 = make_it_categorical_v2(rsv_predictors_df_v2)

rsv_predictors_df_v2.shape

# summary_function_rsv(rsv_predictors_df_v2)

# Extract a reduced sample of the data for modelling
sample_size = 80000
sample_v2_df = rsv_predictors_df_v2.sample(n = sample_size, random_state=42)

sample_v2_df = rsv_predictors_df_v2

In [158]:
selected_features = ['sex', 'marital_status', 'race','patient_regional_location', 'age_group',
                     'Acute_upper_respiratory_infection','Influenza','Pneumonia','Bronchitis','Symptoms_and_signs__digestive_system_and_abdomen','General_symptoms_and_signs','any_symptom',
                     'COPD','AIDS','Asthma_chronic','CCI',
                     'sine','cosine','calendar_year', 
                     'healthcare_seeking', 'influenza_vaccine',
                     'n_symptoms','prev_positive_rsv','previous_test_daydiff','n_immunodeficiencies', 
                     'tumor_indicator','tumor_last_year',
                     'season',
                     'n_tests_that_day']
# selected_features = ['sex', 'marital_status', 'race', 'patient_regional_location', 'age_group',
#                      'Acute_upper_respiratory_infection','Influenza','Pneumonia','Bronchitis','Symptoms_and_signs__digestive_system_and_abdomen','General_symptoms_and_signs','any_symptom',
#                      'COPD','AIDS','Asthma_chronic','CCI',
#                      'sine','cosine','calendar_year', 
#                      'healthcare_seeking', 'influenza_vaccine',
#                      'n_symptoms','prev_positive_rsv','previous_test_daydiff','n_immunodeficiencies', 
#                      'tumor_indicator','tumor_last_year']
selected_features.append('RSV_test_result')

In [161]:
df1 = sample_v2_df[selected_features]

input_test_size = 0.2
random_seed = 42

X_train, y_train, X_test, y_test, sample_weights, preprocessor_rsv = preprocess_and_resample_rsv(
    df1, input_test_size = input_test_size, random_seed = random_seed, resampling_technique = "downsample_upweight", ratio_maj_min = 0.7)

Resampling method chosen:

Downsampling and Upweighting


# 0. Study resampling techniques (WIP)

# 1. Build-up of the models

### 1.1. XGboost
- Train it using the previous approach (GridSearchCV)
- Train it using Bayesian parameter optimization

##### Approach 1: GridSearch CV

In [ ]:
# Approach 1: GridSearch CV
random_seed = 42
cost_sensitive = True

if cost_sensitive:
    weight_dict = {"Negative": 1,
                   "Positive": 50}
    scale_pos_weight = weight_dict["Positive"]/weight_dict["Negative"]  # Use scale_pos_weight parameter
    model_class = XGBClassifier(scale_pos_weight=scale_pos_weight,
                                random_state=random_seed)
else:
    model_class = XGBClassifier(random_state=random_seed)

param_grid = {
    'n_estimators': range(20,205,15),
    'max_depth': range(5,30,1),
    'learning_rate': np.arange(0.01, 0.51, 0.05),
    'min_child_weight': np.arange(1, 11, 1), 
    'gamma': np.arange(0.1, 0.5, 0.1) 
}

target_scorer = make_scorer(f1_score, average='macro')
n_cv_folds = 5

# XGBoost needs labels in numeric format
y_train_numeric = [1 if label == "Positive" else 0 for label in y_train]

model1_xgb = train_model_rsv(model = model_class, param_grid = param_grid, target_scorer = target_scorer, n_cv_folds = n_cv_folds,
                    X_train = X_train, y_train = y_train_numeric)

optimal_threshold = find_optimal_moving_threshold(model = model1_xgb, X_test = X_test, y_test = y_test)
__,__,__,__,__,__,f1 = calculate_performance_metrics_rsv(trained_model = model1_xgb, X_test = X_test, y_test = y_test,
                                                         threshold = optimal_threshold, 
                                                         print_roc = False)

##### Approach 2: Bayesian hyperparameter optimization

In [137]:
# Approach 2: train the model using bayesian hyperparameter optimization

# Scorings = ['accuracy', 'balanced_accuracy', 'f1', 'f1_micro', 'f1_macro', 'f1_weighted', 'precision', 'recall', 'roc_auc']
Scorings = ['f1', 'f1_micro', 'f1_macro', 'f1_weighted', 'recall', 'roc_auc']
y_train_numeric = [1 if label == 'Positive' else 0 for label in y_train]

best_models = {}

for score in tqdm(Scorings):
    classifier = XGBoostClassifier_custom(scoring=score, max_evals=12, sample_weights = sample_weights, cost_sensitive_yn = True)
    classifier.train(X_train, y_train_numeric)
    classifier.predict(X_test, y_test)
    best_models[score] = {'model': classifier.model, 'score_f1': classifier.score_f1, 'score_auc': classifier.score_auc}
    

  0%|          | 0/6 [00:00<?, ?it/s]

--------------------------------------------------------------------
Training XGBoost classifier with objective metric: f1
Tuning Hyperparameters ...


100%|██████████| 12/12 [01:40<00:00,  8.37s/trial, best loss: 0.24443137577723262]
Best Hyperparameters:  {'colsample_bytree': 0.7000000000000001, 'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 5.0, 'min_child_weight': 8.0, 'n_estimators': 5, 'subsample': 0.9}
[18:44:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "verbose" } are not used.

XGBoostClassifier Performance:
Optimal threshold: 0.61
Optimal f1: 0.25958702064896755




 17%|█▋        | 1/6 [01:43<08:38, 103.64s/it]

AUC Score: 0.7557835693275874
Precision / Positive predictive value: 0.4808743169398907
Specificity: 0.9938729442115447
Recall / sensitivity: 0.17777777777777778
Negative predictive value: 0.9742681924511601
Accuracy: 0.968625
F-1: 0.25958702064896755
--------------------------------------------------------------------
Training XGBoost classifier with objective metric: f1_micro
Tuning Hyperparameters ...
100%|██████████| 12/12 [01:06<00:00,  5.58s/trial, best loss: 0.28301860069870644]
Best Hyperparameters:  {'colsample_bytree': 0.7000000000000001, 'gamma': 0.30000000000000004, 'learning_rate': 0.15000000000000002, 'max_depth': 5.0, 'min_child_weight': 2.0, 'n_estimators': 4, 'subsample': 0.7000000000000001}
[18:46:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "verbose" } are not used.

XGBoostClassifier Performance:
Optimal threshold: 0.62
Optimal f1: 0.2424242424242424

...

AUC Score: 0.7531737888397031
Precision / Positive predictive value: 0.2909090909090909
Specificity: 0.982392776523702
Recall / sensitivity: 0.22626262626262628
Negative predictive value: 0.975472302273455
Accuracy: 0.959
F-1: 0.2545454545454545
--------------------------------------------------------------------
Training XGBoost classifier with objective metric: recall
Tuning Hyperparameters ...
100%|██████████| 12/12 [01:30<00:00,  7.50s/trial, best loss: 0.2256535201258819]
Best Hyperparameters:  {'colsample_bytree': 0.7000000000000001, 'gamma': 0.2, 'learning_rate': 0.05, 'max_depth': 10.0, 'min_child_weight': 8.0, 'n_estimators': 4, 'subsample': 0.7000000000000001}
[18:50:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "verbose" } are not used.

XGBoostClassifier Performance:
Optimal threshold: 0.59
Optimal f1: 0.23168654173764905




 83%|████████▎ | 5/6 [07:07<01:25, 85.53s/it]

AUC Score: 0.7549689086935136
Precision / Positive predictive value: 0.7391304347826086
Specificity: 0.9984521122218639
Recall / sensitivity: 0.13737373737373737
Negative predictive value: 0.9731581594166457
Accuracy: 0.9718125
F-1: 0.23168654173764905
--------------------------------------------------------------------
Training XGBoost classifier with objective metric: roc_auc
Tuning Hyperparameters ...
100%|██████████| 12/12 [02:13<00:00, 11.13s/trial, best loss: 0.206069339948042]
Best Hyperparameters:  {'colsample_bytree': 0.8, 'gamma': 0.30000000000000004, 'learning_rate': 0.1, 'max_depth': 10.0, 'min_child_weight': 2.0, 'n_estimators': 3, 'subsample': 0.9}
[18:52:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "verbose" } are not used.

XGBoostClassifier Performance:
Optimal threshold: 0.61
Optimal f1: 0.21403912543153047




100%|██████████| 6/6 [09:28<00:00, 94.76s/it] 

AUC Score: 0.7409889934494901
Precision / Positive predictive value: 0.24866310160427807
Specificity: 0.98187681393099
Recall / sensitivity: 0.18787878787878787
Negative predictive value: 0.9742736464866248
Accuracy: 0.9573125
F-1: 0.21403912543153047


In [136]:
class XGBoostClassifier_custom:

    def __init__(self, scoring, max_evals, cost_sensitive_yn, sample_weights):
        self.scoring = scoring
        self.max_evals = max_evals
        self.cost_sensitive = cost_sensitive_yn
        self.sample_weights = sample_weights
        self.best = None
        self.model = None
        self.score_f1 = None
        self.score_auc = None
        

    def objective(self, space):
        classifier = XGBClassifier(n_estimators = space['n_estimators'],
                                    max_depth = int(space['max_depth']),
                                    learning_rate = space['learning_rate'],
                                    gamma = space['gamma'],
                                    min_child_weight = space['min_child_weight'],
                                    subsample = space['subsample'],
                                    colsample_bytree = space['colsample_bytree'],
                                    )
        # classifier.fit(self.X_train, self.y_train, early_stopping_rounds = space['early_stopping_rounds'], eval_metric = 'logloss',  eval_set=eval_set, verbose=True)
        if self.cost_sensitive:
            classifier.fit(self.X_train, self.y_train, sample_weight = self.sample_weights)
        else: 
            classifier.fit(self.X_train, self.y_train)

        Scores = cross_val_score(estimator = classifier, X = self.X_train, y = self.y_train, cv = 10, scoring=self.scoring)
        score = Scores.mean()
        loss = 1-score
        return {'loss': loss, 'status': STATUS_OK}

    def train(self, X_train, y_train):
        print('--------------------------------------------------------------------')
        print(f'Training XGBoost classifier with objective metric: {self.scoring}')
        self.X_train = X_train
        self.y_train = y_train

        self.space = {
        'max_depth' : hp.quniform('max_depth',  5, 21, 5),
        'learning_rate' : hp.quniform('learning_rate', 0.010, 0.200, 0.050),
        'n_estimators' : hp.choice('n_estimators', range(20, 205, 25)),
        'gamma' : hp.quniform('gamma', 0, 0.50, 0.1),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 2),
        'subsample' : hp.quniform('subsample', 0.6, 1, 0.1),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.6, 1.0, 0.1),
        'early_stopping_rounds': 100
        }

        trials = Trials()
        print("Tuning Hyperparameters ...")
        self.best = fmin(fn=self.objective,
                    space=self.space,
                    algo=tpe.suggest,
                    max_evals=self.max_evals,
                    trials=trials)
        print("Best Hyperparameters: ", self.best)
        self.fit_model()

    def fit_model(self):
        self.model = XGBClassifier(n_estimators = self.best['n_estimators'],
                                max_depth = int(self.best['max_depth']),
                                learning_rate = self.best['learning_rate'],
                                gamma = self.best['gamma'],
                                min_child_weight = self.best['min_child_weight'],
                                subsample = self.best['subsample'],
                                colsample_bytree = self.best['colsample_bytree'], 
                                verbose = True
                                )
        # self.model.fit(self.X_train, self.y_train, early_stopping_rounds = self.space['early_stopping_rounds'], eval_metric = 'logloss')
        if self.cost_sensitive:
            self.model.fit(self.X_train, self.y_train, sample_weight = self.sample_weights)
        else: 
            self.model.fit(self.X_train, self.y_train)

        print('XGBoostClassifier Performance:')

        # Scores_f1 = cross_val_score(estimator = self.model, X = self.X_train, y = self.y_train, cv = 10, scoring='f1')
        # self.score_f1 = Scores_f1.mean()
        # print("Train Set 10-Fold F1-Score: ", self.score)

        # Scores_auc = cross_val_score(estimator = self.model, X = self.X_train, y = self.y_train, cv = 10, scoring='roc_auc')
        # self.score_auc = Scores_auc.mean()
        # print("Train Set 10-Fold F1-Score: ", self.score)

    def predict(self, X_test, y_test):

        optimal_threshold = find_optimal_moving_threshold(model = self.model, X_test = X_test, y_test = y_test)
        self.score_auc,__,__,__,__,__,self.score_f1 = calculate_performance_metrics_rsv(trained_model = self.model, X_test = X_test, y_test = y_test,
                                                         threshold = optimal_threshold, 
                                                         print_roc = False)

## 1.2. LightGBM

As the Bayesian hyperparameter has proven more effective, this approach will be followed here too

In [155]:
class LightGBMClassifier_custom:

    def __init__(self, scoring, max_evals, cost_sensitive_yn, sample_weights):
        self.scoring = scoring
        self.max_evals = max_evals
        self.cost_sensitive = cost_sensitive_yn
        self.sample_weights = sample_weights
        self.best = None
        self.model = None
        self.score_f1 = None
        self.score_auc = None
        
    def objective(self, space):
        classifier = lgbm.LGBMClassifier(n_estimators = space['n_estimators'],
                                    max_depth = int(space['max_depth']),
                                    learning_rate = space['learning_rate'],
                                    min_child_weight = space['min_child_weight'],
                                    subsample = space['subsample'],
                                    colsample_bytree = space['colsample_bytree'],
                                    )
        
        if self.cost_sensitive:
            classifier.fit(self.X_train, self.y_train, sample_weight = self.sample_weights)
        else: 
            classifier.fit(self.X_train, self.y_train)

        Scores = cross_val_score(estimator = classifier, X = self.X_train, y = self.y_train, cv = 10, scoring=self.scoring)
        score = Scores.mean()
        loss = 1-score
        return {'loss': loss, 'status': STATUS_OK}

    def train(self, X_train, y_train):
        print('--------------------------------------------------------------------')
        print(f'Training LightGBM classifier with objective metric: {self.scoring}')
        self.X_train = X_train
        self.y_train = y_train

        self.space = {
        'max_depth' : hp.quniform('max_depth', 5, 21, 5),
        'learning_rate' : hp.uniform('learning_rate', 0.010, 0.200),
        'n_estimators' : hp.choice('n_estimators', range(20, 205, 25)),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 2),
        'subsample' : hp.quniform('subsample', 0.6, 1, 0.1),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.6, 1.0, 0.1),
        }

        trials = Trials()
        print("Tuning Hyperparameters ...")
        self.best = fmin(fn=self.objective,
                    space=self.space,
                    algo=tpe.suggest,
                    max_evals=self.max_evals,
                    trials=trials)
        print("Best Hyperparameters: ", self.best)
        self.fit_model()

    def fit_model(self):
        self.model = lgbm.LGBMClassifier(n_estimators = self.best['n_estimators'],
                                max_depth = int(self.best['max_depth']),
                                learning_rate = self.best['learning_rate'],
                                min_child_weight = self.best['min_child_weight'],
                                subsample = self.best['subsample'],
                                colsample_bytree = self.best['colsample_bytree']
                                )

        if self.cost_sensitive:
            self.model.fit(self.X_train, self.y_train, sample_weight = self.sample_weights)
        else: 
            self.model.fit(self.X_train, self.y_train)

        print('LightGBMClassifier Performance:')
        # These should be part of the evaluation set, not the training set

        # Scores_f1 = cross_val_score(estimator = self.model, X = self.X_train, y = self.y_train, cv = 10, scoring='f1')
        # self.score_f1 = Scores_f1.mean()
        # print("Train Set 10-Fold F1-Score: ", self.score_f1)

        # Scores_auc = cross_val_score(estimator = self.model, X = self.X_train, y = self.y_train, cv = 10, scoring='roc_auc')
        # self.score_auc = Scores_auc.mean()
        # print("Train Set 10-Fold AUC Score: ", self.score_auc)

    def predict(self, X_test, y_test):       
        optimal_threshold = find_optimal_moving_threshold(model = self.model, X_test = X_test, y_test = y_test)
        self.score_auc,__,__,__,__,__,self.score_f1 = calculate_performance_metrics_rsv(trained_model = self.model, X_test = X_test, y_test = y_test, threshold = optimal_threshold, print_roc = False)


In [162]:
# Scorings = ['accuracy', 'balanced_accuracy', 'f1', 'f1_micro', 'f1_macro', 'f1_weighted', 'precision', 'recall', 'roc_auc']
Scorings = ['f1', 'f1_micro', 'f1_macro', 'f1_weighted', 'recall', 'roc_auc']
y_train_numeric = [1 if label == 'Positive' else 0 for label in y_train]

best_models = {}

for score in tqdm(Scorings):
    classifier = LightGBMClassifier_custom(scoring=score, max_evals=12, sample_weights = sample_weights, cost_sensitive_yn = True)
    classifier.train(X_train, y_train_numeric)
    classifier.predict(X_test, y_test)
    best_models[score] = {'model': classifier.model, 'score_f1': classifier.score_f1, 'score_auc': classifier.score_auc}

  0%|          | 0/6 [00:00<?, ?it/s]

--------------------------------------------------------------------
Training LightGBM classifier with objective metric: f1
Tuning Hyperparameters ...
100%|██████████| 12/12 [00:52<00:00,  4.35s/trial, best loss: 0.21952563377577106]
Best Hyperparameters:  {'colsample_bytree': 0.8, 'learning_rate': 0.07340009969395415, 'max_depth': 15.0, 'min_child_weight': 2.0, 'n_estimators': 2, 'subsample': 0.9}
LightGBMClassifier Performance:
Optimal threshold: 0.56
Optimal f1: 0.35348837209302325




 17%|█▋        | 1/6 [00:58<04:54, 58.93s/it]

AUC Score: 0.7702883429854845
Precision / Positive predictive value: 1.0
Specificity: 1.0
Recall / sensitivity: 0.21468926553672316
Negative predictive value: 0.9756111825944555
Accuracy: 0.9757727167092726
F-1: 0.35348837209302325
--------------------------------------------------------------------
Training LightGBM classifier with objective metric: f1_micro
Tuning Hyperparameters ...
100%|██████████| 12/12 [00:46<00:00,  3.90s/trial, best loss: 0.2704347491535858]
Best Hyperparameters:  {'colsample_bytree': 0.8, 'learning_rate': 0.1174101408036662, 'max_depth': 5.0, 'min_child_weight': 6.0, 'n_estimators': 3, 'subsample': 0.9}
LightGBMClassifier Performance:
Optimal threshold: 0.5700000000000001
Optimal f1: 0.3764705882352941




 33%|███▎      | 2/6 [01:51<03:40, 55.16s/it]

...

AUC Score: 0.792338532364991
Precision / Positive predictive value: 1.0
Specificity: 1.0
Recall / sensitivity: 0.21468926553672316
Negative predictive value: 0.9756111825944555
Accuracy: 0.9757727167092726
F-1: 0.35348837209302325
--------------------------------------------------------------------
Training LightGBM classifier with objective metric: recall
Tuning Hyperparameters ...
100%|██████████| 12/12 [00:40<00:00,  3.41s/trial, best loss: 0.15591947913898474]
Best Hyperparameters:  {'colsample_bytree': 0.8, 'learning_rate': 0.03858693138737751, 'max_depth': 20.0, 'min_child_weight': 4.0, 'n_estimators': 1, 'subsample': 0.9}
LightGBMClassifier Performance:
Optimal threshold: 0.52
Optimal f1: 0.35348837209302325




 83%|████████▎ | 5/6 [04:17<00:49, 49.71s/it]

AUC Score: 0.7533068453785112
Precision / Positive predictive value: 1.0
Specificity: 1.0
Recall / sensitivity: 0.21468926553672316
Negative predictive value: 0.9756111825944555
Accuracy: 0.9757727167092726
F-1: 0.35348837209302325
--------------------------------------------------------------------
Training LightGBM classifier with objective metric: roc_auc
Tuning Hyperparameters ...
100%|██████████| 12/12 [00:51<00:00,  4.30s/trial, best loss: 0.1948260836524922]
Best Hyperparameters:  {'colsample_bytree': 0.8, 'learning_rate': 0.13886733343361768, 'max_depth': 5.0, 'min_child_weight': 4.0, 'n_estimators': 3, 'subsample': 0.7000000000000001}
LightGBMClassifier Performance:
Optimal threshold: 0.6
Optimal f1: 0.3708206686930091




100%|██████████| 6/6 [05:15<00:00, 52.54s/it]

AUC Score: 0.7871131391974653
Precision / Positive predictive value: 0.9606299212598425
Specificity: 0.9997002577783106
Recall / sensitivity: 0.2297551789077213
Negative predictive value: 0.976060872110038
Accuracy: 0.9759470137113642
F-1: 0.3708206686930091


## 2. Ensemble of the models


In [ ]:
def MetaClassifier(Algorithms, X_train, y_train, X_test, y_test):
    """
    Function to build a soft-voting ensemble meta-classifier
    Arguments:
    Algorithms -- list of model iterations outputs of different algorithms
    train_dict -- a dictionary of train set features and labels
    test_dict (optional) -- a dictionary that contains test set features and labels
    Return:
    out -- a dictionary of meta-classifier model ('BestModel'), a dataframe of performance metrics of all models (Metrics), 
           and a dataframe of metaclassifier predicted labels and probabilities on trainset instances (train_df_pred)
    """

    estimators=[]
    weights = []
    results = []

    train_dict = {
        'X_train': X_train,
        'y_train': y_train 
    }
    test_dict = {
        'X_test ': X_test,
        'y_test': y_test
    }


    print('Generating a Soft-Voting Ensemble Classifier...')

    for alg in tqdm(Algorithms):
        for metric in tqdm(list(alg.keys())):
            # for voting ensemble
            estimators.append((str(alg)+'_'+metric, alg[metric]['model']))
            weights.append(alg[metric]['score_f1'])

            # for results
            results.append(eval_model(model=alg[metric]['model'], train_dict=train_dict, test_dict=test_dict))

    results_df = pd.DataFrame(results)

    results_df.sort_values(by='f1', ascending=False)

    # keeping top five iterations for the ensemble
    KeepIdx = results_df.nlargest(5, 'f1').index

    BestEstimators= [estimators[i] for i in KeepIdx.tolist()]
    BestWeights = [weights[i] for i in KeepIdx.tolist()]

    X_train, y_train = train_dict['X_train'], train_dict['y_train']

    MetaClassifier = VotingClassifier(estimators=BestEstimators, voting='soft', weights=BestWeights)
    MetaClassifier = MetaClassifier.fit(X_train, y_train)

    results_df = results_df.append(eval_model(MetaClassifier, train_dict, test_dict), ignore_index=True).sort_values(by='train_CV_f1', ascending=False) # add metaclassification model metrics

    # storing metaclassifier performance on trainset
    y_pred = MetaClassifier.predict(X_train)
    y_proba =  MetaClassifier.predict_proba(X_train)
    train_df_pred = X_train.copy()
    train_df_pred['y_true'] = y_train
    train_df_pred['y_hat'] = y_pred
    train_df_pred['y_hat_proba'] = y_proba[:,1] # probability of belonging to class 1

    out = {'BestModel': BestEstimators[0][1], 'Metrics':results_df, 'train_df_pred':train_df_pred}
    return out